In [207]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import sys
import numpy as np
import pandas as pd
import regex as re
import requests
import random

In [208]:
chromedriver = "/Users/I511581/Documents" # path to the chromedriver executable
chromedriver = os.path.expanduser(chromedriver)
print('chromedriver path: {}'.format(chromedriver))
sys.path.append(chromedriver)
driver = webdriver.Chrome(executable_path="/Users/I511581/chromedriver")

chromedriver path: /Users/I511581/Documents


In [209]:
url = "https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E85312&maxBedrooms=3&minBedrooms=2&maxPrice=800000&minPrice=600000&radius=0.25&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords="
driver.get(url)

In [210]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
listings = soup.find_all("a", class_="propertyCard-link property-card-updates")
listings[:5]

[<a class="propertyCard-link property-card-updates" data-test="property-details" href="/properties/120052040#/?channel=RES_BUY"><address class="propertyCard-address property-card-updates" itemprop="address"><meta content="Portway, Stratford, London, E15" itemprop="streetAddress"/><meta content="GB" itemprop="addressCountry"/>Portway, Stratford, London, E15</address><div class="property-information"><span class="text">House</span><span class="seperator">|</span><span class="no-svg-bed-icon bed-icon"><svg viewbox="0 -2 16 14"><title>3 bedrooms</title><use xlink:href="#core-icon--bed-icon"></use></svg></span><span aria-hidden="true" class="text">3</span><span class="seperator">|</span><span class="no-svg-bathroom-icon bathroom-icon"><svg viewbox="0 0 14 16"><title>1 bathroom</title><use xlink:href="#core-icon--bathroom-icon"></use></svg></span><span aria-hidden="true" class="text">1</span></div></a>,
 <a class="propertyCard-link property-card-updates" data-test="property-details" href="/p

In [212]:
def main():

    # initialise index, this tracks the page number we are on. every additional page adds 24 to the index

    # create lists to store our data
    all_apartment_links = []
    all_description = []
    all_address = []
    all_price = []


    index = 0
    for pages in range(5):
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"
        }

        if index == 0:
            rightmove = f"https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E85312&&maxBedrooms=3&minBedrooms=2&maxPrice=800000&minPrice=600000&radius=0.25&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords="
        elif index != 0:
            rightmove = f"https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E85312&&maxBedrooms=3&minBedrooms=2&maxPrice=800000&minPrice=600000&radius=0.25&index={index}&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords="

        # request our webpage
        res = requests.get(rightmove, headers=headers)
        # check status
        res.raise_for_status()

        # This gets the list of apartments
        apartments = soup.find_all("div", class_="l-searchResult is-list")

        # This gets the number of listings
        number_of_listings = soup.find("span", {"class": "searchHeader-resultCount"})
        number_of_listings = number_of_listings.get_text()
        number_of_listings = int(number_of_listings.replace(",", ""))

        for i in range(len(apartments)):
            # tracks which apartment we are on in the page
            first_var = apartments[i]

            # append link
            apartment_info = first_var.find("a", class_="propertyCard-link")
            link = "https://www.rightmove.co.uk" + apartment_info.attrs["href"]
            all_apartment_links.append(link)

            # append address
            address = (
                apartment_info.find("address", class_="propertyCard-address")
                .get_text()
                .strip()
            )
            all_address.append(address)

            # append description
            description = (
                apartment_info.find("h2", class_="propertyCard-title")
                .get_text()
                .strip()
            )
            all_description.append(description)

            # append price
            price = (
                first_var.find("div", class_="propertyCard-priceValue")
                .get_text()
                .strip()
            )
            all_price.append(price)


            print(f"You have scrapped {pages + 1} pages of apartment listings.")
            print(f"You have {number_of_listings - index} listings left to go")
            print("\n")

        # Between each request, set up a random delay between 1 and 3 seconds
        time.sleep(random.randint(1,3))
        index = index + 24
        if index >= number_of_listings:
            break

    # convert data to dataframe
    data = {
        "Links": all_apartment_links,
        "Address": all_address,
        "Description": all_description,
        "Price": all_price,
    }

    df = pd.DataFrame.from_dict(data)
    df.to_csv(r"rightmove_scraper.csv", encoding="utf-8", header="true", index=False)


if __name__ == "__main__":
    main()

AttributeError: 'NoneType' object has no attribute 'get_text'